In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import nltk

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

from nltk.corpus import stopwords
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
import nltk

[nltk_data] Downloading package wordnet to /home/tiziri-
[nltk_data]     tamani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/tiziri-
[nltk_data]     tamani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/tiziri-tamani/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/tiziri-
[nltk_data]     tamani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
df = pd.read_csv('scitweets_export.tsv', sep='\t')
df.head()

,Unnamed: 0,tweet_id,text,science_related,scientific_claim,scientific_reference,scientific_context
0,0,"3,16669998137483E+017",Knees are a bit sore. i guess that's a sign th...,0,0.0,0.0,0.0
1,1,"3,19090866545386E+017",McDonald's breakfast stop then the gym 🏀💪,0,0.0,0.0,0.0
2,2,"3,22030931022066E+017",Can any Gynecologist with Cancer Experience ex...,1,1.0,0.0,0.0
3,3,"3,22694830620807E+017",Couch-lock highs lead to sleeping in the couch...,1,1.0,0.0,0.0
4,4,"3,28524426658329E+017",Does daily routine help prevent problems with ...,1,1.0,0.0,0.0


In [24]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

# Initialisation (à mettre en début de notebook)
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
custom_stop_words = {"http", "https", "rt", "co", "amp", "via"}
stop_words.update(custom_stop_words)
lemmatizer = WordNetLemmatizer()

def preprocess_tweet(tweet):
    # 1. Supprime mentions (@) et caractères spéciaux
    tweet = re.sub(r"@\w+|\W", " ", tweet)
    # 2. Convertit en minuscules
    tweet = tweet.lower()
    # 3. Tokenization et lemmatisation
    tokens = word_tokenize(tweet)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # 4. Recombine en texte
    return " ".join(tokens)

[nltk_data] Downloading package stopwords to /home/tiziri-
[nltk_data]     tamani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/tiziri-
[nltk_data]     tamani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/tiziri-
[nltk_data]     tamani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
# ------------------------------------------------------------
# ÉTAPE 2 : {CLAIM, REF} vs {CONTEXT} (Classification binaire)
# ------------------------------------------------------------

# 1. Préparation des données
sci_df = df[df['science_related'] == 1].copy()

# Nettoyage du texte SI NECESSAIRE
if 'cleaned_text' not in sci_df.columns:
    sci_df['cleaned_text'] = sci_df['text'].apply(preprocess_tweet)

# Création de la target
sci_df['claim_or_ref'] = ((sci_df['scientific_claim'] == 1) | (sci_df['scientific_reference'] == 1)).astype(int)

# 2. Vectorisation TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=20000)
X = vectorizer.fit_transform(sci_df['cleaned_text'])
y = sci_df['claim_or_ref']

# 3. Équilibrage SMOTE
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# 4. Split des données
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

# 5. Entraînement des modèles (exemple avec Random Forest)
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# 6. Évaluation
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.94      0.97        64
           1       0.95      1.00      0.97        73

    accuracy                           0.97       137
   macro avg       0.97      0.97      0.97       137
weighted avg       0.97      0.97      0.97       137



In [26]:
print(sci_df[['text', 'cleaned_text', 'claim_or_ref']].head())

                                                text  \
2  Can any Gynecologist with Cancer Experience ex...   
3  Couch-lock highs lead to sleeping in the couch...   
4  Does daily routine help prevent problems with ...   
6  “Traffic Jam” In Brain’s Neurons Could Be Caus...   
7  Can playing more games improve lives and save ...   

                                        cleaned_text  claim_or_ref  
2  gynecologist cancer experience explain danger ...             1  
3  couch lock high lead sleeping couch got ta sto...             1  
4  daily routine help prevent problem bipolar dis...             1  
6  traffic jam brain neuron could cause statin re...             1  
7  playing game improve life save world uetvu099a...             1  


In [27]:
# ------------------------------------------------------------
# ÉTAPE 2 : Classification Binaire Multi-Label {CLAIM/REF} et {CONTEXT}
# ------------------------------------------------------------

stop_words = set(stopwords.words('english'))
custom_stop_words = {"http", "https", "rt", "co", "amp", "via"}
stop_words.update(custom_stop_words)
lemmatizer = WordNetLemmatizer()

def preprocess_tweet(tweet):
    # 1. Supprime mentions (@) et caractères spéciaux
    #tweet = re.sub(r"@\w+|\W", " ", tweet)
    # 2. Convertit en minuscules
    tweet = tweet.lower()
    # 3. Tokenization et lemmatisation
    tokens = word_tokenize(tweet)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # 4. Recombine en texte
    return " ".join(tokens)


# 1. Préparation des données (tweets scientifiques uniquement)
sci_df = df[df['science_related'] == 1].copy()

# 2. Création des DEUX colonnes cibles (peuvent valoir 1 simultanément)
sci_df['is_claim_or_ref'] = ((sci_df['scientific_claim'] == 1) | (sci_df['scientific_reference'] == 1)).astype(int)
sci_df['is_context'] = (sci_df['scientific_context'] == 1).astype(int)

# Afficher les combinaisons possibles
print("Distribution des combinaisons :")
print(sci_df[['is_claim_or_ref', 'is_context']].value_counts())

# 3. Prétraitement du texte (si pas déjà fait)
if 'cleaned_text' not in sci_df.columns:
    sci_df['cleaned_text'] = sci_df['text'].apply(preprocess_tweet)

# 4. Vectorisation TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=20000)
X = vectorizer.fit_transform(sci_df['cleaned_text'])

# 5. Création des cibles multi-labels
y = sci_df[['is_claim_or_ref', 'is_context']].values

# 6. Utilisation d'un modèle adapté au multi-label (ex: ClassifierChain + RandomForest)
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier

model = ClassifierChain(
    RandomForestClassifier(random_state=42),
    order='random',
    random_state=42
)

# 7. Évaluation par validation croisée
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

y_pred = cross_val_predict(model, X, y, cv=10)
print("\nRapport de classification pour chaque label :")
print(classification_report(y, y_pred, target_names=['CLAIM/REF', 'CONTEXT']))

from sklearn.metrics import accuracy_score

# Accuracy multi-label stricte (exact match ratio)
exact_match_accuracy = accuracy_score(y, y_pred)
print(f"\nAccuracy stricte (exact match) : {exact_match_accuracy:.4f}")


Distribution des combinaisons :
is_claim_or_ref  is_context
1                1             218
                 0             124
0                1              33
Name: count, dtype: int64

Rapport de classification pour chaque label :
              precision    recall  f1-score   support

   CLAIM/REF       0.91      1.00      0.95       342
     CONTEXT       0.75      0.97      0.85       251

   micro avg       0.84      0.99      0.91       593
   macro avg       0.83      0.99      0.90       593
weighted avg       0.84      0.99      0.91       593
 samples avg       0.85      0.99      0.89       593


Accuracy stricte (exact match) : 0.6880


In [28]:
#pip install imbalanced-learn
# Exemple de tweets avec prédictions
sample_results = sci_df.copy()
sample_results[['pred_claim_ref', 'pred_context']] = y_pred
print(sample_results[['text', 'is_claim_or_ref', 'is_context', 'pred_claim_ref', 'pred_context']].head(10))

                                                 text  is_claim_or_ref  \
2   Can any Gynecologist with Cancer Experience ex...                1   
3   Couch-lock highs lead to sleeping in the couch...                1   
4   Does daily routine help prevent problems with ...                1   
6   “Traffic Jam” In Brain’s Neurons Could Be Caus...                1   
7   Can playing more games improve lives and save ...                1   
8   The effect of climate change on iceberg produc...                1   
14  @RepCohen @SenAlexander @SenBobCorker pls supp...                0   
15  Poverty is the greatest cause behind child lab...                1   
16  "@TheFactsBook: Drinking chocolate milk has be...                1   
24  A wise Physician said, “The best medicine for ...                1   

    is_context  pred_claim_ref  pred_context  
2            0             1.0           0.0  
3            0             1.0           0.0  
4            0             1.0           1.0

In [29]:
# ------------------------------------------------------------
# ÉTAPE 2 : Classification Binaire Multi-Label - Version Corrigée
# ------------------------------------------------------------

from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, f1_score
from imblearn.ensemble import BalancedRandomForestClassifier
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.multioutput import MultiOutputClassifier


stop_words = set(stopwords.words('english'))
custom_stop_words = {"http", "https", "rt", "co", "amp", "via"}
stop_words.update(custom_stop_words)
lemmatizer = WordNetLemmatizer()

def preprocess_tweet(tweet):
    # 1. Supprime mentions (@) et caractères spéciaux
    #tweet = re.sub(r"@\w+|\W", " ", tweet)
    # 2. Convertit en minuscules
    tweet = tweet.lower()
    # 3. Tokenization et lemmatisation
    tokens = word_tokenize(tweet)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # 4. Recombine en texte
    return " ".join(tokens)


# 1. Préparation des données (tweets scientifiques uniquement)
sci_df = df[df['science_related'] == 1].copy()

# 2. Création des DEUX colonnes cibles (peuvent valoir 1 simultanément)
sci_df['is_claim_or_ref'] = ((sci_df['scientific_claim'] == 1) | (sci_df['scientific_reference'] == 1)).astype(int)
sci_df['is_context'] = (sci_df['scientific_context'] == 1).astype(int)

# Afficher les combinaisons possibles
print("Distribution des combinaisons :")
print(sci_df[['is_claim_or_ref', 'is_context']].value_counts())

# 3. Prétraitement du texte (si pas déjà fait)
if 'cleaned_text' not in sci_df.columns:
    sci_df['cleaned_text'] = sci_df['text'].apply(preprocess_tweet)

# 4. Vectorisation TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=20000)
X = vectorizer.fit_transform(sci_df['cleaned_text'])

# 5. Création des cibles multi-labels
y = sci_df[['is_claim_or_ref', 'is_context']].values


# Fonction de scoring personnalisée pour multi-label
def multilabel_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

# Configuration KFold
kf = KFold(n_splits=10, shuffle=True, random_state=42)
scorer = make_scorer(multilabel_accuracy)

# Dictionnaire des modèles révisé
models = {
    "SVM": {
        "model": SVC(probability=True),
        "params": {
            'estimator__C': [0.1, 1, 10],
            'estimator__kernel': ['linear', 'rbf']
        }
    },
    "Naive Bayes": {
        "model": MultinomialNB(),
        "params": {
            'estimator__alpha': [0.1, 0.5, 1.0],
            'estimator__fit_prior': [True, False]
        }
    },
    "k-NN": {
        "model": KNeighborsClassifier(),
        "params": {
            'estimator__n_neighbors': [3, 5, 7],
            'estimator__weights': ['uniform', 'distance'],
            'estimator__metric': ['euclidean', 'cosine']
        }
    },
    "Random Forest": {
        "model": RandomForestClassifier(random_state=42),
        "params": {
            'estimator__n_estimators': [50, 100],
            'estimator__max_depth': [None, 10, 20],
            'estimator__min_samples_split': [2, 5]
        }
    },
    "Logistic Regression": {
        "model": LogisticRegression(max_iter=1000),
        "params": {
            'estimator__C': [0.1, 1, 10],
            'estimator__penalty': ['l1', 'l2'],
            'estimator__solver': ['liblinear']
        }
    },
    "Gradient Boosting": {
        "model": GradientBoostingClassifier(random_state=42),
        "params": {
            'estimator__n_estimators': [50, 100],
            'estimator__learning_rate': [0.01, 0.1],
            'estimator__max_depth': [3, 5]
        }
    }
}

# Évaluation des modèles
for name, config in models.items():
    print(f"\n{'='*60}")
    print(f"Optimisation de {name}")
    print(f"{'='*60}")
    
    # Création du pipeline
    mo = MultiOutputClassifier(config['model'])
    
    # GridSearch avec validation croisée interne
    grid = GridSearchCV(
        estimator=mo,
        param_grid=config['params'],
        cv=10,
        scoring=scorer,
        n_jobs=-1,
        verbose=1
    )
    
    # IMPORTANT: On doit fit le GridSearch avant d'accéder aux best_params_
    grid.fit(X, y)  # <-- Cette ligne était manquante
    
    # Cross-validation externe
    cv_scores = cross_val_score(
        grid.best_estimator_,
        X,
        y,
        cv=kf,
        scoring=scorer,
        n_jobs=-1
    )
    
    print(f"\nMeilleurs paramètres: {grid.best_params_}")
    print(f"Accuracy (moyenne ± écart-type): {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
    print(f"Scores par fold: {cv_scores}")

Distribution des combinaisons :
is_claim_or_ref  is_context
1                1             218
                 0             124
0                1              33
Name: count, dtype: int64

Optimisation de SVM
Fitting 10 folds for each of 6 candidates, totalling 60 fits

Meilleurs paramètres: {'estimator__C': 10, 'estimator__kernel': 'linear'}
Accuracy (moyenne ± écart-type): 0.8094 ± 0.0427
Scores par fold: [0.73684211 0.82894737 0.80263158 0.81578947 0.82894737 0.83783784
 0.82432432 0.89189189 0.75675676 0.77027027]

Optimisation de Naive Bayes
Fitting 10 folds for each of 6 candidates, totalling 60 fits

Meilleurs paramètres: {'estimator__alpha': 0.5, 'estimator__fit_prior': True}
Accuracy (moyenne ± écart-type): 0.8053 ± 0.0415
Scores par fold: [0.75       0.81578947 0.81578947 0.78947368 0.86842105 0.7972973
 0.81081081 0.87837838 0.78378378 0.74324324]

Optimisation de k-NN
Fitting 10 folds for each of 12 candidates, totalling 120 fits

Meilleurs paramètres: {'estimator__metri

/home/tiziri-tamani/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/tiziri-tamani/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/tiziri-tamani/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/tiziri-tamani/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/tiziri-tamani/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/tiziri-tamani/anaconda3/lib/python3.12/site-packages/s


Meilleurs paramètres: {'estimator__C': 10, 'estimator__penalty': 'l1', 'estimator__solver': 'liblinear'}
Accuracy (moyenne ± écart-type): 0.8627 ± 0.0330
Scores par fold: [0.80263158 0.89473684 0.86842105 0.85526316 0.86842105 0.89189189
 0.86486486 0.91891892 0.82432432 0.83783784]

Optimisation de Gradient Boosting
Fitting 10 folds for each of 8 candidates, totalling 80 fits

Meilleurs paramètres: {'estimator__learning_rate': 0.1, 'estimator__max_depth': 3, 'estimator__n_estimators': 50}
Accuracy (moyenne ± écart-type): 0.8721 ± 0.0489
Scores par fold: [0.80263158 0.90789474 0.88157895 0.85526316 0.88157895 0.93243243
 0.82432432 0.95945946 0.81081081 0.86486486]


In [30]:
# ------------------------------------------------------------
# ÉTAPE 2 : Classification Multi-Label avec Équilibrage Adapté
# ------------------------------------------------------------

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.preprocessing import MultiLabelBinarizer

## 1. Prétraitement des données (identique à votre version)
stop_words = set(stopwords.words('english'))
custom_stop_words = {"http", "https", "rt", "co", "amp", "via"}
stop_words.update(custom_stop_words)
lemmatizer = WordNetLemmatizer()

def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tokens = word_tokenize(tweet)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

## 2. Préparation des données
sci_df = df[df['science_related'] == 1].copy()
sci_df['cleaned_text'] = sci_df['text'].apply(preprocess_tweet)

# Création des labels combinés
sci_df['labels'] = sci_df.apply(lambda row: 
                               ['claim_or_ref'] if row['is_claim_or_ref'] else [] + 
                               ['context'] if row['is_context'] else [], axis=1)

# Transformation multi-label
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(sci_df['labels'])
X = vectorizer.fit_transform(sci_df['cleaned_text'])

## 3. Équilibrage Adapté (Solution pour contourner la limitation d'imblearn)
def balanced_train_test_split(X, y, test_size=0.2):
    # Séparation stratifiée
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y.sum(axis=1), random_state=42
    )
    
    # Rééchantillonnage manuel pour l'entraînement
    label_counts = y_train.sum(axis=0)
    max_count = max(label_counts)
    
    resampled_indices = []
    for label_idx in range(y.shape[1]):
        label_indices = np.where(y_train[:, label_idx] == 1)[0]
        resampled_indices.extend(
            np.random.choice(label_indices, size=max_count, replace=True))
        
    X_train_res = X_train[resampled_indices]
    y_train_res = y_train[resampled_indices]
    
    return X_train_res, X_test, y_train_res, y_test

X_train, X_test, y_train, y_test = balanced_train_test_split(X, y)

## 4. Modèle et Évaluation
model = MultiOutputClassifier(
    RandomForestClassifier(
        n_estimators=200,
        max_depth=15,
        class_weight='balanced',
        random_state=42
    )
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Rapport de Classification:")
print(classification_report(y_test, y_pred, target_names=mlb.classes_))

## 5. Optimisation avec GridSearchCV
param_grid = {
    'estimator__n_estimators': [100, 200],
    'estimator__max_depth': [10, 15, None],
    'estimator__min_samples_split': [2, 5]
}

grid = GridSearchCV(
    model, 
    param_grid, 
    cv=5,
    scoring='f1_micro',
    n_jobs=-1,
    verbose=1
)
grid.fit(X_train, y_train)

print(f"\nMeilleurs paramètres: {grid.best_params_}")
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)
print("\nPerformance du meilleur modèle:")
print(classification_report(y_test, y_pred, target_names=mlb.classes_))

KeyError: 'is_claim_or_ref'